# Universidad de los Andes

Proyecto presentado por:
* Javier Camilo Garcia Matos
* Daniel Felipe Caro Torres

## Proyecto: Claficador de texto según las 17 ODS

El objetivo del proyecto es desarrollar una solución, basada en técnicas de procesamiento del
lenguaje natural y machine learning, que permita clasificar automáticamente un texto en los
17 ODS, ofreciendo una forma de presentación de resultados a través de una herramienta de
fácil comprensión para el usuario final.

### Planteamiento del problema y solucion

En el marco de los Objetivos de Desarrollo Sostenible (ODS) de la ONU, muchas organizaciones buscan clasificar grandes volúmenes de texto para entender qué partes de la información corresponden a cada objetivo: fin de la pobreza, salud y bienestar, educación de calidad, igualdad de género, etc. Para ello, se pueden usar técnicas de procesamiento de lenguaje natural (NLP) y machine learning, evitando el análisis manual de miles de documentos.

###### Este notebook presentará el desarrollo paso a paso de un clasificador entrenado en textos en español, siguiendo el flujo:

1. Importación y exploración de datos
2. Separación de train/test/validación
3. Preprocesamiento (limpieza, stopwords, stemming, etc.)
4. Vectorización (TF-IDF)
5. Reducción de dimensionalidad (SVD)
6. Entrenamiento y selección de modelo
7. Evaluación (métricas y ejemplos concretos en un conjunto de prueba)
8. Conclusiones

Se finaliza mostrando la clasificación de algunos ejemplos de test, verificando la utilidad del modelo.

#### 1. Importación y exploración de datos

In [76]:
import nltk
import numpy as np
import pandas as pd
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer # SnowballStemmer para espa;ol

# entrenar/validar el modelo
from sklearn.model_selection import train_test_split

# Representación de textos
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Reducción de dimensionalidad
from sklearn.decomposition import TruncatedSVD

#-----------------

# Clasificadores 
from sklearn.linear_model import LogisticRegression

# Para métricas de evaluación
from sklearn.metrics import classification_report, confusion_matrix

# Para pipeline y búsqueda de hiperparámetros
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

nltk.download('stopwords')  


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

##### Carga y exploración de datos

In [77]:
df = pd.read_excel('Train_textosODS.xlsx')
df.sample(5)

,textos,ODS
3214,Las organizaciones de mujeres de la sociedad c...,5
9316,Los Estados miembros de la Unión Europea se ha...,16
7521,ResumenLas reglas de jurisdicción e inmunidad ...,16
6609,"En la mayoría de los países, la redistribución...",10
4264,Este capítulo explora cómo una red de defensor...,16


In [78]:
print('Tamaño de los datos:',df.shape)

Tamaño de los datos: (9656, 2)


In [79]:
print('Cantidad de datos nulos:')
print(df.isna().sum())

Cantidad de datos nulos:
textos    0
ODS       0
dtype: int64


In [80]:
print('Cantidad de datos duplicados:')
print(df.duplicated().sum())

Cantidad de datos duplicados:
0


#### 2. Separación de train/test/validación

In [81]:
df_train, df_test = train_test_split(df, test_size = 0.2, random_state=0)
print('Cantidad de datos de entrenamiento:',len(df_train))
print('Cantidad de datos de prueba:',len(df_test))



Cantidad de datos de entrenamiento: 7724
Cantidad de datos de prueba: 1932


In [82]:
x_train = df_train['textos']
y_train = df_train['ODS']

x_test = df_test['textos']
y_test = df_test['ODS']

#### 3. Preprocesamiento (limpieza, stopwords, stemming, etc.)

In [83]:
def text_preprocess(text):
    tokenizer = RegexpTokenizer(r'\w+')
    stemmer = nltk.SnowballStemmer('spanish')  # Stemming en español
    spanish_stops = set(stopwords.words('spanish'))
    
    # minúsculas
    text = text.lower()

    tokens = tokenizer.tokenize(text)
    # remover stopwords
    tokens = [word for word in tokens if word not in spanish_stops]
    # stemming
    tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(tokens)



#### 4. Vectorización (TF-IDF)

In [84]:
vectorizer = TfidfVectorizer(preprocessor=text_preprocess)

#### 5. Reducción de dimensionalidad (SVD)

Usamos TruncatedSVD para reduccoin de dimensionalidad, ya que lo consideramos más adecuado para datos dispersos, como los generados por TF-IDF, ya que no requiere centrar los datos y puede trabajar eficientemente con matrices dispersas.

In [85]:
tsvd = TruncatedSVD(n_components=100, random_state=42)

#### 6. Entrenamiento y selección de modelo

Escogemos Regresión Logística para multiclase:

In [86]:
clf = LogisticRegression(
    multi_class='multinomial',
    solver='saga', 
    max_iter=1000,
    random_state=42
)


In [89]:
full_pipeline = Pipeline([
    ('tfidf', vectorizer),
    ('svd', tsvd),
    ('clf', clf)
])


predecir en validación/test

##### Búsqueda de hiperparámetros 

In [ ]:
param_grid = {
    'svd__n_components': [50, 100, 200],
    'clf__C': [0.01, 0.1, 1, 10, 100]
}

grid_search = GridSearchCV(
    full_pipeline, 
    param_grid, 
    scoring='accuracy', 
    cv=5,
    verbose=1
)

grid_search.fit(x_train, y_train)
print("Mejores hiperparámetros:", grid_search.best_params_)
best_model = grid_search.best_estimator_


Fitting 5 folds for each of 15 candidates, totalling 75 fits


#### 7. Evaluación (métricas y ejemplos concretos en un conjunto de prueba)

In [ ]:
print("Evaluación en conjunto de prueba:")
y_test_pred = best_model.predict(x_test)
print(classification_report(y_test, y_test_pred))


In [ ]:
cm = confusion_matrix(y_test, y_test_pred)
print("Matriz de confusión:\n", cm)


##### Ejemplos concretos de prediccion

In [ ]:
import random

sample_indices = random.sample(list(df_test.index), 4)
for idx in sample_indices:
    raw_text = df_test.loc[idx, 'textos']
    true_label = df_test.loc[idx, 'ODS']
    pred_label = best_model.predict([raw_text])[0]
    
    print("="*60)
    print(f"Texto:\n{raw_text}\n")
    print(f"ODS real: {true_label}")
    print(f"ODS predicho: {pred_label}")
    print("="*60, "\n")


#### 8. Conclusiones

##### 1) Preparacion de datos y reduccion de al dimensionalidad:

 - Se ha justificado el uso de TF-IDF por resaltar términos relevantes 
     y SVD por condensar la información en menos dimensiones (50, 100 o 200), 
     mejorando tiempo de entrenamiento y reduciendo ruido.

##### 2) Resultados del modelo

El modelo alcanzó un desempeño destacado al clasificar automáticamente textos en relación con los distintos ODS. En promedio, la exactitud rondó alrededor del 86–87% en el conjunto de prueba, lo cual indicó que la mayoría de las clases fueron reconocidas con buenos valores de precisión, recall y f1-score. Sin embargo, se evidenció que ciertos ODS con menos ejemplos disponibles presentaron resultados más modestos, lo que sugirió la necesidad de recopilar más datos o ajustar algunas fases del preprocesamiento para mejorar aún más la robustez del modelo.

Asimismo, la reducción de dimensionalidad mediante SVD contribuyó a una mayor eficiencia y a evitar el sobreajuste en un espacio inicial muy extenso producto de la representación TF-IDF. Esto, sumado a la búsqueda de hiperparámetros a través de GridSearchCV, permitió afinar los valores óptimos para la regresión logística, logrando un equilibrio entre complejidad y rendimiento. En última instancia, el uso de métricas globales y de ejemplos concretos demostró la solidez del enfoque para clasificar grandes volúmenes de texto en español, brindando una base confiable para su aplicación en contextos reales de análisis y toma de decisiones sobre los ODS.


##### 3) Ejemplos reales:

   - Se mostraron 4 textos del conjunto de prueba, junto a la 
     etiqueta real y la predicción, evidenciando la utilidad del modelo 
     para clasificar nuevos documentos.